In [1]:
# Try to import, if it fails prompt to install
from __future__ import annotations
import os
import json
import hashlib
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass
from tqdm import tqdm
import random
import pickle
from dotenv import load_dotenv

load_dotenv()

# OpenAI library imports
import asyncio
import openai  # Good for catching error types like openai.RateLimitError
from openai import OpenAI
from openai import AsyncOpenAI
import tiktoken

# Basic libraries
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_curve,
    precision_recall_curve,
    f1_score,
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import torch

# Visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns

import os, time, hashlib, json, random
from typing import List, Tuple

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report

# Transformers
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    AutoTokenizer,
    AutoModel,
)
from sentence_transformers import SentenceTransformer

# Text processing
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Text Scrambler Library
try:
    import nlpaug.augmenter.word as naw
    import nlpaug.augmenter.sentence as nas
except ImportError:
    print("nlpaug not found, installing...")
    os.system("pip install nlpaug")
    import nlpaug.augmenter.word as naw
    import nlpaug.augmenter.sentence as nas

# Transformers and sentence-transformers

from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    GPT2LMHeadModel,
    GPT2Tokenizer,
)
from sentence_transformers import SentenceTransformer
from dataclasses import dataclass, field
from typing import List, Optional

# NLTK data download
import nltk

print("Downloading required NLTK data...")
nltk.download("punkt_tab")
nltk.download("punkt", quiet=True)
nltk.download("averaged_perceptron_tagger", quiet=True)
nltk.download("averaged_perceptron_tagger_eng", quiet=True)
nltk.download("wordnet", quiet=True)
nltk.download("omw-1.4", quiet=True)
nltk.download("stopwords", quiet=True)
print("NLTK data download completed.")
import logging

# Configuration log
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


@dataclass
class DetectionConfig:
    """Detecting Configuration Classes"""

    revision_model: str = "t5-small"
    embedding_model: str = "all-MiniLM-L6-v2"

    api_key: Optional[str] = field(default_factory=lambda: os.getenv("OPENAI_API_KEY"))
    base_url: Optional[str] = field(
        default_factory=lambda: os.getenv("OPENAI_BASE_URL")
    )

    perturbation_rate: float = 0.15

    perturbation_methods: List[str] = field(
        default_factory=lambda: ["synonym", "contextual"]
    )

    similarity_threshold: float = 0.95
    use_ml_classifier: bool = True

    batch_size: int = 16
    max_length: int = 512

    def __post_init__(self):
        if self.perturbation_methods is None:
            self.perturbation_methods = ["synonym", "contextual", "backtranslation"]


class EnhancedTextPerturber:

    def __init__(self, config: DetectionConfig):
        self.config = config
        self._init_augmenters()

    def _init_augmenters(self):

        self.augmenters = {
            "synonym": naw.SynonymAug(
                aug_src="wordnet", aug_p=self.config.perturbation_rate
            ),
            "contextual": naw.ContextualWordEmbsAug(
                model_path="distilbert-base-uncased",
                action="substitute",
                aug_p=self.config.perturbation_rate,
            ),
            "random_swap": naw.RandomWordAug(
                action="swap", aug_p=self.config.perturbation_rate
            ),
            "spelling": naw.SpellingAug(aug_p=self.config.perturbation_rate),
        }

    def perturb(self, text: str, method: Optional[str] = None) -> str:
        """
        Intelligent Perturbation Strategies

        Args.
            text: original text
            method: specified method, None is chosen randomly
        Returns.
            Scrambled text
        """
        if method is None:
            method = np.random.choice(self.config.perturbation_methods)

        try:
            if method == "backtranslation":
                return self._backtranslate(text)
            elif method in self.augmenters:
                augmented = self.augmenters[method].augment(text)
                return augmented[0] if isinstance(augmented, list) else augmented
            else:

                return self._mixed_perturbation(text)
        except Exception as e:
            print(f"Perturbation failed: {e}")
            return self._simple_perturb(text)

    def _mixed_perturbation(self, text: str) -> str:

        sentences = sent_tokenize(text)
        perturbed_sentences = []

        for sent in sentences:

            method = np.random.choice(list(self.augmenters.keys()))
            try:
                perturbed = self.augmenters[method].augment(sent)
                perturbed_sent = (
                    perturbed[0] if isinstance(perturbed, list) else perturbed
                )
                perturbed_sentences.append(perturbed_sent)
            except:
                perturbed_sentences.append(sent)

        return " ".join(perturbed_sentences)

    def _simple_perturb(self, text: str) -> str:

        words = text.split()

        num_changes = max(1, int(len(words) * self.config.perturbation_rate))
        indices = np.random.choice(
            range(len(words)), size=min(num_changes, len(words)), replace=False
        )

        for idx in indices:

            word = words[idx]
            if len(word) > 3:
                words[idx] = word[:-1] + np.random.choice(list("aeiou"))

        return " ".join(words)

    def _backtranslate(self, text: str) -> str:

        return text


class EnhancedLLMReviser:
    """Enhanced LLM Rewriter"""

    def __init__(self, config: DetectionConfig):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.client = None
        self.model = None
        self.tokenizer = None
        self._init_reviser()

    def _init_model(self):
        model_name = self.config.revision_model
        """Initialising the rewrite model"""
        if self.config.revision_model.startswith("t5"):

            self.tokenizer = T5Tokenizer.from_pretrained(self.config.revision_model)
            self.model = T5ForConditionalGeneration.from_pretrained(
                self.config.revision_model
            )
        elif self.config.revision_model == "gpt2":
            # GPT-2 as an option
            self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
            self.model = GPT2LMHeadModel.from_pretrained("gpt2")
            self.tokenizer.pad_token = self.tokenizer.eos_token
        elif model_name.startswith("gpt-"):
            # Initialising the OpenAI API client
            api_key = self.config.api_key or os.getenv("OPENAI_API_KEY")
            if not api_key:
                raise ValueError(
                    "Please provide api_key in DetectionConfig or set the OPENAI_API_KEY environment variable."
                )

            self.client = OpenAI(api_key=api_key, base_url=self.config.base_url)
            self.max_retries = 3
            self.retry_delay = 1.0
            print(f"OpenAI client initialized for model: {model_name}")
            if self.config.base_url:
                print(f"Using custom base URL: {self.config.base_url}")
        else:
            print(
                f"Warning: Unknown revision model '{model_name}'. Will use rule-based fallback."
            )

        self.model.to(self.device)
        self.model.eval()

    def _init_reviser(self):
        """Initialise rewrite model or API client according to configuration"""
        model_name = self.config.revision_model

        if model_name.startswith("t5"):
            self.tokenizer = T5Tokenizer.from_pretrained(model_name)
            self.model = T5ForConditionalGeneration.from_pretrained(model_name)
            self.model.to(self.device).eval()
            print(f"Local T5 model loaded: {model_name}")

        elif model_name == "gpt2":
            self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
            self.model = GPT2LMHeadModel.from_pretrained(model_name)
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.model.to(self.device).eval()
            print(f"Local GPT-2 model loaded: {model_name}")

        elif model_name.startswith("gpt-"):
            # Initialising the OpenAI API client
            api_key = self.config.api_key or os.getenv("OPENAI_API_KEY")
            if not api_key:
                raise ValueError(
                    "Please provide api_key in DetectionConfig or set the OPENAI_API_KEY environment variable."
                )

            # self.client = OpenAI(api_key=api_key, base_url=self.config.base_url)
            self.client = AsyncOpenAI(
                api_key=api_key, base_url=self.config.base_url
            )  # Using Asynchronous Clients
            self.max_retries = 3
            self.retry_delay = 1.0
            print(f"OpenAI client initialized for model: {model_name}")
            if self.config.base_url:
                print(f"Using custom base URL: {self.config.base_url}")
        else:
            print(
                f"Warning: Unknown revision model '{model_name}'. Will use rule-based fallback."
            )

    # def revise(self, text: str, cache: Dict[str, str]) -> str:
    async def revise(
        self, original_text: str, perturbed_text: str, cache: Dict[str, str]
    ) -> str:  # change to async def
        """
        Rewriting Text with LLM

        Args.
            original_text: text to be rewritten
            cache: Cached dictionary
        Returns.
            Rewritten text
        """
        # 生成缓存键
        cache_key = hashlib.md5(
            f"{original_text}_{self.config.revision_model}".encode()
        ).hexdigest()

        # Check if the result for the original text is already in the cache
        if cache_key in cache:
            return cache[cache_key]

        # print(f"\norigin text: {original_text}")

        try:
            model_name = self.config.revision_model
            revised = ""
            if model_name.startswith("t5") or model_name == "gpt2":
                if model_name.startswith("t5"):
                    revised = self._revise_with_t5(perturbed_text)
                    # print(f"t5 rewrite: {revised}")
                else:  # gpt2
                    revised = self._revise_with_gpt2(perturbed_text)
                    # print(f"gpt2 rewrite: {revised}")
            elif model_name.startswith("gpt-"):
                # api rewrite
                # revised = self._revise_with_api(perturbed_text)
                revised = await self._revise_with_api(perturbed_text)  # change to await
                # print(f"gpt3.5 rewrite: {revised}")
            else:
                revised = self._rule_based_revision(perturbed_text)

            cache[cache_key] = revised
            return revised

        except Exception as e:
            print(f"Revision failed with model {self.config.revision_model}: {e}")
            fallback_revision = self._rule_based_revision(perturbed_text)
            cache[cache_key] = fallback_revision
            return fallback_revision

    def _revise_with_t5(self, text: str) -> str:
        """Rewrite using the T5 model"""

        prompt = f"paraphrase: {text}"

        inputs = self.tokenizer.encode(
            prompt,
            return_tensors="pt",
            max_length=512,  # Maximum length of input truncation
            truncation=True,
        ).to(self.device)

        with torch.no_grad():

            outputs = self.model.generate(
                inputs,
                num_beams=4,
                early_stopping=True,
                max_length=256,
                min_length=40,
                repetition_penalty=1.2,
            )

        revised = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return revised

    def _revise_with_gpt2(self, text: str) -> str:
        """Rewrite using the GPT-2 model"""
        try:
            prompt_templates = [
                f"Paraphrase the following text while keeping the same meaning: {text}\n\nParaphrased version:",
                f"Rewrite this sentence in a different way: {text}\n\nRewritten:",
                f"Express this differently: {text}\n\nAlternative expression:",
            ]
            prompt = np.random.choice(prompt_templates)
            inputs = self.tokenizer.encode(
                prompt,
                return_tensors="pt",
                max_length=512,
                truncation=True,
                padding=True,  # Ensure padding is enabled to handle the mask
            )
            input_ids = inputs.input_ids.to(self.device)
            attention_mask = inputs.attention_mask.to(self.device)

            with torch.no_grad():
                outputs = self.model.generate(
                    input_ids=input_ids,  # Pass input_ids explicitly
                    attention_mask=attention_mask,
                    max_length=min(len(input_ids[0]) + 100, 1024),
                    num_return_sequences=1,
                    temperature=0.8,
                    do_sample=True,
                    top_p=0.9,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    no_repeat_ngram_size=3,
                    # early_stopping=True,
                )

            #
            full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            rewritten = ""
            for marker in [
                "Paraphrased version:",
                "Rewritten:",
                "Alternative expression:",
            ]:
                if marker in full_text:
                    rewritten = full_text.split(marker)[-1].strip()
                    break

            if not rewritten:
                rewritten = full_text[len(prompt) :].strip()

            if "\n" in rewritten:
                rewritten = rewritten.split("\n")[0]
            if "." in rewritten:
                rewritten = rewritten.split(".")[0] + "."

            return rewritten if rewritten else text

        except Exception as e:
            print(f"  Fixed Rewrite Failure: {e}")
            return text

    # def _revise_with_api(self, text: str) -> str:
    async def _revise_with_api(self, text: str) -> str:  # change to async def
        """Use OpenAI API with better prompting strategy"""
        if not self.client:
            raise ValueError("OpenAI client not initialized.")

        # Better prompt that encourages standardization
        system_prompt = """You are a professional editor. Your task is to rewrite the given text to make it more formal, standardized, and academic in style. 

Key guidelines:
- Replace casual expressions with formal language
- Use precise technical terms where appropriate
- Maintain professional tone throughout
- Standardize sentence structure
- Remove colloquialisms and personal expressions
- Keep the core meaning but express it in a more refined way

The more casual or informal the original text, the more changes you should make."""

        user_prompt = f"Please rewrite this text in a formal, academic style:\n\n{text}"

        for attempt in range(self.max_retries):
            try:
                response = await self.client.chat.completions.create(  # change to await
                    model=self.config.revision_model,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt},
                    ],
                    temperature=0.3,  # Lower temperature for more consistent output
                    top_p=0.9,
                )
                return response.choices[0].message.content.strip()

            except Exception as e:
                print(f"API error (attempt {attempt + 1}): {e}")
                if attempt < self.max_retries - 1:
                    await asyncio.time.sleep(
                        self.retry_delay
                    )  # Using asynchronous sleep
                else:
                    return text

        return text

    def _rule_based_revision(self, text: str) -> str:

        import re

        sentences = sent_tokenize(text)
        if len(sentences) > 2:

            sentences = self._reorder_sentences(sentences)

        revised_sentences = []
        for sent in sentences:

            synonyms = {
                "important": "significant",
                "however": "nevertheless",
                "therefore": "thus",
                "because": "since",
                "many": "numerous",
                "show": "demonstrate",
            }

            for word, syn in synonyms.items():
                sent = re.sub(r"\b" + word + r"\b", syn, sent, flags=re.IGNORECASE)

            revised_sentences.append(sent)

        return " ".join(revised_sentences)

    def _reorder_sentences(self, sentences: List[str]) -> List[str]:
        """Intelligent Sentence Rearrangement"""
        # Keep the first and last sentences
        if len(sentences) <= 2:
            return sentences

        first = sentences[0]
        last = sentences[-1]
        middle = sentences[1:-1]

        # break up the middle of a sentence
        np.random.shuffle(middle)

        return [first] + middle + [last]


class MultiFeatureExtractor:
    """Multi-dimensional feature extractor"""

    def __init__(self, config: DetectionConfig):
        self.config = config
        self.sentence_model = SentenceTransformer(config.embedding_model)

        if self.sentence_model.tokenizer.pad_token is None:
            print(
                f"Warning: Tokenizer for '{config.embedding_model}' is missing a pad token. Setting it to eos_token."
            )
            self.sentence_model.tokenizer.pad_token = (
                self.sentence_model.tokenizer.eos_token
            )
        # --- Add these lines to view parameters ---
        print(f"\n--- Parameters for Embedding Model: {config.embedding_model} ---")
        # Print the model architecture (all layers)
        print(self.sentence_model)

        # Calculate and print the total number of trainable parameters
        total_params = sum(
            p.numel() for p in self.sentence_model.parameters() if p.requires_grad
        )
        print(f"Total Trainable Parameters: {total_params:,}")
        self._init_linguistic_features()

    def _init_linguistic_features(self):
        """Initialised Linguistic Feature Extraction"""
        try:
            nltk.download("punkt", quiet=True)
            nltk.download("averaged_perceptron_tagger", quiet=True)
            nltk.download("stopwords", quiet=True)
            self.stop_words = set(stopwords.words("english"))
        except:
            self.stop_words = set()

    def extract_features(self, original: str, revised: str) -> Dict[str, float]:
        """
        Extract multi-dimensional features

        Returns:
            feature dictionary
        """
        features = {}

        # 1. semantic similarity
        features["semantic_similarity"] = self._compute_semantic_similarity(
            original, revised
        )

        # 2. lexical level feature
        features.update(self._extract_lexical_features(original, revised))

        # 3. Syntactic features
        features.update(self._extract_syntactic_features(original, revised))

        # 4. style
        features.update(self._extract_stylistic_features(original, revised))

        # 5. Edit Distance Characteristics
        features.update(self._extract_edit_features(original, revised))

        return features

    def _compute_semantic_similarity(self, text1: str, text2: str) -> float:
        """Calculating semantic similarity"""
        emb1 = self.sentence_model.encode(text1)
        emb2 = self.sentence_model.encode(text2)

        from sklearn.metrics.pairwise import cosine_similarity

        return float(cosine_similarity([emb1], [emb2])[0, 0])

    def _extract_lexical_features(
        self, original: str, revised: str
    ) -> Dict[str, float]:
        """Extraction of vocabulary-level features"""
        orig_words = set(word_tokenize(original.lower()))
        rev_words = set(word_tokenize(revised.lower()))

        # overlap rate of vocabulary
        overlap = len(orig_words & rev_words)
        total = len(orig_words | rev_words)

        features = {
            "word_overlap_ratio": overlap / total if total > 0 else 0,
            "vocabulary_change_ratio": (
                len(orig_words ^ rev_words) / total if total > 0 else 0
            ),
            "unique_words_ratio": (
                len(rev_words - orig_words) / len(rev_words) if rev_words else 0
            ),
        }

        # Change in proportion of deactivated words
        orig_stop = len([w for w in orig_words if w in self.stop_words])
        rev_stop = len([w for w in rev_words if w in self.stop_words])

        features["stopword_ratio_change"] = abs(
            (orig_stop / len(orig_words) if orig_words else 0)
            - (rev_stop / len(rev_words) if rev_words else 0)
        )

        return features

    def _extract_syntactic_features(
        self, original: str, revised: str
    ) -> Dict[str, float]:
        """Extracting syntactic features"""
        orig_sents = sent_tokenize(original)
        rev_sents = sent_tokenize(revised)

        features = {
            "sentence_count_ratio": (
                len(rev_sents) / len(orig_sents) if orig_sents else 1
            ),
            "avg_sentence_length_change": (
                abs(
                    np.mean([len(word_tokenize(s)) for s in orig_sents])
                    - np.mean([len(word_tokenize(s)) for s in rev_sents])
                )
                if orig_sents and rev_sents
                else 0
            ),
        }

        try:
            orig_pos = nltk.pos_tag(word_tokenize(original))
            rev_pos = nltk.pos_tag(word_tokenize(revised))

            # Calculate the percentage change in the main lexical categories
            for pos_type in [
                "NN",
                "VB",
                "JJ",
                "RB",
            ]:  # Nouns, verbs, adjectives, adverbs
                orig_count = sum(1 for _, pos in orig_pos if pos.startswith(pos_type))
                rev_count = sum(1 for _, pos in rev_pos if pos.startswith(pos_type))

                features[f"pos_{pos_type}_ratio_change"] = abs(
                    (orig_count / len(orig_pos) if orig_pos else 0)
                    - (rev_count / len(rev_pos) if rev_pos else 0)
                )
        except:
            pass

        return features

    def _extract_stylistic_features(
        self, original: str, revised: str
    ) -> Dict[str, float]:
        """Extracting stylistic features"""
        features = {}

        # Changes in the use of punctuation
        orig_punct = sum(1 for c in original if c in ".,!?;:")
        rev_punct = sum(1 for c in revised if c in ".,!?;:")

        features["punctuation_ratio_change"] = abs(
            (orig_punct / len(original) if original else 0)
            - (rev_punct / len(revised) if revised else 0)
        )

        # Change in proportion of capital letters
        orig_caps = sum(1 for c in original if c.isupper())
        rev_caps = sum(1 for c in revised if c.isupper())

        features["capitalization_ratio_change"] = abs(
            (orig_caps / len(original) if original else 0)
            - (rev_caps / len(revised) if revised else 0)
        )

        # Change in average word length
        orig_words = word_tokenize(original)
        rev_words = word_tokenize(revised)

        features["avg_word_length_change"] = (
            abs(
                np.mean([len(w) for w in orig_words])
                - np.mean([len(w) for w in rev_words])
            )
            if orig_words and rev_words
            else 0
        )

        return features

    def _extract_edit_features(self, original: str, revised: str) -> Dict[str, float]:
        """Extraction of edit distance related features"""
        from difflib import SequenceMatcher

        # Character-level similarity
        char_similarity = SequenceMatcher(None, original, revised).ratio()

        # word-level similarity
        orig_words = word_tokenize(original)
        rev_words = word_tokenize(revised)
        word_similarity = SequenceMatcher(None, orig_words, rev_words).ratio()

        features = {
            "char_level_similarity": char_similarity,
            "word_level_similarity": word_similarity,
            "length_ratio": len(revised) / len(original) if original else 1,
        }

        return features


class AITextDetector:
    """Main detector class"""

    def __init__(self, config: DetectionConfig = None):
        self.config = config or DetectionConfig()
        self.perturber = EnhancedTextPerturber(self.config)
        self.reviser = EnhancedLLMReviser(self.config)
        self.feature_extractor = MultiFeatureExtractor(self.config)
        self.classifier = None
        self.scaler = StandardScaler()
        self.cache = {}
        self.api_concurrency_limit = asyncio.Semaphore(10)

    # In the AITextDetector class

    async def detect_batch(
        self, texts: List[str], labels: Optional[List[int]] = None
    ) -> Dict[str, np.ndarray]:
        """
        Asynchronously detects text in a batch, with robust caching and parallel API calls.
        """
        all_features = []
        all_scores = []

        if not texts:
            # Handle the case where an empty list is passed
            return {
                "predictions": np.array([]),
                "probabilities": np.array([]),
                "features": np.array([]),
                "similarity_scores": np.array([]),
            }

        print("Processing texts...")

        # Step 1: Perform synchronous, CPU-bound perturbation first.
        perturbed_texts = [
            self.perturber.perturb(text) for text in tqdm(texts, desc="Perturbing")
        ]

        # Step 2: Define a helper function to manage semaphore and call the new revise method.
        # This helper will be used to create our list of concurrent tasks.
        async def revise_with_limit(original_text, perturbed_text):
            async with self.api_concurrency_limit:
                # Call the updated revise function with both original and perturbed text
                return await self.reviser.revise(
                    original_text, perturbed_text, self.cache
                )

        # Step 3: Create a list of NEW coroutine tasks.
        # Each task is a call to our helper function with a pair of original and perturbed texts.
        print(f"Revising texts with {self.reviser.config.revision_model}...")
        revision_tasks = [
            revise_with_limit(orig, pert) for orig, pert in zip(texts, perturbed_texts)
        ]

        # Step 4: Concurrently run all revision tasks using asyncio.gather.
        # This preserves the order of the results and is wrapped in tqdm for a progress bar.
        all_revised_texts = await asyncio.gather(
            *tqdm(revision_tasks, desc="Revising", total=len(revision_tasks))
        )

        # Step 5: Process the results. This part is synchronous again.
        print("Extracting features...")
        for i, original_text in enumerate(texts):
            # The order is preserved, so we can safely pair original and revised texts.
            revised_text = all_revised_texts[i]
            features = self.feature_extractor.extract_features(
                original_text, revised_text
            )
            all_features.append(features)
            all_scores.append(features.get("semantic_similarity", 0.5))

        # --- The rest of the function remains the same ---
        feature_matrix = pd.DataFrame(all_features).fillna(0).values

        if self.config.use_ml_classifier:
            if labels is not None and self.classifier is None:
                self._train_classifier(feature_matrix, labels)

            if self.classifier is not None:
                feature_matrix_scaled = self.scaler.transform(feature_matrix)
                predictions = self.classifier.predict(feature_matrix_scaled)
                probabilities = self.classifier.predict_proba(feature_matrix_scaled)[
                    :, 1
                ]
            else:
                # Fallback if classifier isn't trained
                predictions = (
                    np.array(all_scores) > self.config.similarity_threshold
                ).astype(int)
                probabilities = np.array(all_scores)
        else:
            # Logic for when the ML classifier is turned off
            predictions = (
                np.array(all_scores) > self.config.similarity_threshold
            ).astype(int)
            probabilities = np.array(all_scores)

        return {
            "predictions": predictions,
            "probabilities": probabilities,
            "features": feature_matrix,
            "similarity_scores": np.array(all_scores),
        }

    def _train_classifier(self, features: np.ndarray, labels: List[int]):
        """Training Machine Learning Classifiers"""
        print("Training classifier...")

        # Standardised features
        features_scaled = self.scaler.fit_transform(features)

        # Using Random Forests
        self.classifier = RandomForestClassifier(
            n_estimators=100, max_depth=10, random_state=42, n_jobs=-1
        )

        print(f"\n--- RandomForestClassifier Parameters ---")
        print(self.classifier.get_params())

        self.classifier.fit(features_scaled, labels)

        # Characteristic importance
        feature_names = list(pd.DataFrame(features).columns)
        importances = self.classifier.feature_importances_

        print("\nTop 10 Most Important Features:")
        for feat, imp in sorted(
            zip(feature_names, importances), key=lambda x: x[1], reverse=True
        )[:10]:
            print(f"  {feat}: {imp:.4f}")

    def save_model(self, path: str):
        """Save the model"""
        import pickle

        model_data = {
            "classifier": self.classifier,
            "scaler": self.scaler,
            "config": self.config,
            "cache": self.cache,
        }

        with open(path, "wb") as f:
            pickle.dump(model_data, f)

        print(f"Model saved to {path}")

    def load_model(self, path: str):
        """Loading Models"""
        import pickle

        with open(path, "rb") as f:
            model_data = pickle.load(f)

        self.classifier = model_data["classifier"]
        self.scaler = model_data["scaler"]
        self.config = model_data["config"]
        self.cache = model_data["cache"]

        print(f"Model loaded from {path}")


class CustomDataLoader:
    """Data Loader Class"""

    def __init__(self, data_dir: str = "./data"):
        self.data_dir = data_dir
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)
            logger.info(f"Created data directory: {data_dir}")

    def load_data(self, path: str) -> pd.DataFrame:
        """
        Reads a dataset with 'text' and 'label' columns and returns a DataFrame.

        Args.
            path: path to the data file, 'finance' means use financial dataset, 'sample' means create sample data.
        Returns.
            df: columns = ['id', 'text', 'label']
        """
        if path == "finance":
            return self.load_finance_dataset()
        if path == "sample":
            return self.create_sample_dataset()
        elif path == "test":
            return self.load_origin_dataset()

        # If it is a relative path, add a data catalogue prefix
        if not os.path.isabs(path) and not os.path.exists(path):
            path = os.path.join(self.data_dir, path)

        try:
            if path.endswith(".csv"):
                df = pd.read_csv(path)
            elif path.endswith(".jsonl"):
                df = pd.read_json(path, lines=True)
            elif path.endswith(".json"):
                df = pd.read_json(path)
            else:
                raise ValueError(f"Unsupported file format: {path}")

            # Ensure that the necessary columns are available
            if "text" not in df.columns:
                raise ValueError("Missing 'text' column in dataset")
            if "label" not in df.columns:
                raise ValueError("Missing 'label' column in dataset")

            # If there is no id column, automatically generate
            if "id" not in df.columns:
                df["id"] = range(len(df))

            # Data Cleaning
            df = df.dropna(subset=["text", "label"])
            df = df[df["text"].str.len() > 10]  # Filtering text that is too short

            logger.info(f"Loaded {len(df)} samples from {path}")
            logger.info(f"Label distribution: {df['label'].value_counts().to_dict()}")

            return df[["id", "text", "label"]]

        except FileNotFoundError:
            logger.warning(f"File {path} not found. Creating sample dataset...")
            return self.create_sample_dataset()

    def find_finance_data_files(self) -> Dict[str, str]:
        """Find financial dataset files"""
        logger.info("Searching for finance data files...")

        # Local environment path
        possible_paths = [
            self.data_dir,
            os.path.join(self.data_dir, "finance"),
            os.path.join(self.data_dir, "finance-dataset"),
            "./finance_data",
            ".",
            "./data",
        ]

        files_found = {}

        for path in possible_paths:
            if os.path.exists(path):
                logger.debug(f"Checking path: {path}")
                for root, dirs, files in os.walk(path):
                    for file in files:
                        if "revised_human_finance" in file or "revised_human" in file:
                            files_found["revised_human"] = os.path.join(root, file)
                        elif (
                            "revised_chatgpt_finance" in file
                            or "revised_chatgpt" in file
                        ):
                            files_found["revised_chatgpt"] = os.path.join(root, file)
                        elif "finance.jsonl" in file:
                            files_found["original"] = os.path.join(root, file)

        return files_found

    def load_finance_dataset(self) -> pd.DataFrame:
        """
        Load financial dataset - use revised text as data

        Returns.
            df: columns = ['id', 'text', 'label']
            where text is the revised text, label: 0=human, 1=chatgpt
        """
        files = self.find_finance_data_files()

        logger.info("Loading finance datasets...")
        logger.info(f"Found files: {list(files.keys())}")

        all_texts = []
        all_labels = []

        if "revised_human" in files:
            try:
                logger.info(
                    f"Loading revised human texts from: {files['revised_human']}"
                )
                with open(files["revised_human"], "r", encoding="utf-8") as f:
                    for line_no, line in enumerate(f, 1):
                        line = line.strip()
                        if line:
                            try:

                                item = json.loads(line)
                                for idx, text in item.items():

                                    cleaned_text = (
                                        text.strip()
                                        .replace("\\n\\n", " ")
                                        .replace("\\n", " ")
                                    )
                                    if len(cleaned_text) > 20:
                                        all_texts.append(cleaned_text)
                                        all_labels.append(0)  # 0 = human
                            except json.JSONDecodeError as e:
                                logger.warning(f"  Line {line_no} parse error: {e}")
                logger.info(
                    f"  Loaded {len([l for l in all_labels if l == 0])} human texts"
                )
            except Exception as e:
                logger.error(f"Error loading revised human texts: {e}")

        # Load revised ChatGPT text (one JSON object per line)
        if "revised_chatgpt" in files:
            try:
                logger.info(
                    f"Loading revised ChatGPT texts from: {files['revised_chatgpt']}"
                )
                with open(files["revised_chatgpt"], "r", encoding="utf-8") as f:
                    for line_no, line in enumerate(f, 1):
                        line = line.strip()
                        if line:
                            try:

                                item = json.loads(line)
                                for idx, text in item.items():

                                    cleaned_text = (
                                        text.strip()
                                        .replace("\\n\\n", " ")
                                        .replace("\\n", " ")
                                    )
                                    if len(cleaned_text) > 20:
                                        all_texts.append(cleaned_text)
                                        all_labels.append(1)  # 1 = chatgpt
                            except json.JSONDecodeError as e:
                                logger.warning(f"  Line {line_no} parse error: {e}")
                logger.info(
                    f"  Loaded {len([l for l in all_labels if l == 1])} ChatGPT texts"
                )
            except Exception as e:
                logger.error(f"Error loading revised ChatGPT texts: {e}")

        # Creating a DataFrame
        df = pd.DataFrame(
            {"id": range(len(all_texts)), "text": all_texts, "label": all_labels}
        )

        # Disrupt the data
        df = df.sample(frac=1, random_state=42).reset_index(drop=True)
        df["id"] = range(len(df))

        logger.info(f"\n=== Dataset Summary ===")
        logger.info(f"Total samples: {len(df)}")
        logger.info(f"Human texts: {len(df[df['label'] == 0])}")
        logger.info(f"ChatGPT texts: {len(df[df['label'] == 1])}")

        # Show Example
        if len(df) > 0:
            logger.info("\nSample texts:")
            for label in [0, 1]:
                label_name = "Human" if label == 0 else "ChatGPT"
                samples = df[df["label"] == label]
                if len(samples) > 0:
                    sample_text = samples.iloc[0]["text"]
                    logger.info(f"[{label_name}]: {sample_text[:100]}...")

        return df[["id", "text", "label"]]

    def create_sample_dataset(self) -> pd.DataFrame:
        """
        Create a sample dataset for testing

        Returns:
            df: columns = ['id', 'text', 'label']
        """
        sample_data = {
            "id": range(6),
            "text": [
                "There is most likely an error in the WSJ's data.  Yahoo! Finance reports the PE on the Russell 2000 to be 15 as of 83111 and SP 500 PE to be 13 (about the same as WSJ). Good catch, though!  E-mail WSJ, perhaps they will be grateful.",
                "I know this question has a lot of answers already, but I feel the answers are phrased either strongly against, or mildly for, co-signing. What it amounts down to is that this is a personal choice. You cannot receive reliable information as to whether or not co-signing this loan is a good move due to lack of information. The person involved is going to know the person they would be co-signing for, and the people on this site will only have their own personal preferences of experiences to draw from. You know if they are reliable, if they will be able to pay off the loan without need for the banks to come after you.  This site can offer general theories, but I think it should be kept in mind that this is wholly a personal decision for the person involved, and them alone to make based on the facts that they know and we do not.",
                "I think the best investment strategy is to diversify your portfolio across different asset classes.",
                "Historical price-to-earnings (PE) ratios for small-cap and large-cap stocks can vary significantly over time and may not be directly comparable due to the different characteristics of these two categories of stocks.Small-cap stocks, which are defined as stocks with a market capitalization of less than $2 billion, tend to be riskier and more volatile than large-cap stocks, which have a market capitalization of $10 billion or more. As a result, investors may be willing to pay a higher price for the potential growth opportunities offered by small-cap stocks, which can lead to higher PE ratios.On the other hand, large-cap stocks tend to be more established and stable, with a longer track record of earnings and revenue growth. As a result, these stocks may trade at lower PE ratios, as investors may be less willing to pay a premium for their growth potential.It is important to note that PE ratios are just one factor to consider when evaluating a stock and should not be used in isolation. Other factors, such as the company's financial health, industry trends, and macroeconomic conditions, can also impact a stock's PE ratio.",
                "Co-signing a personal loan for a friend or family member can be a risky proposition. When you co-sign a loan, you are agreeing to be responsible for the loan if the borrower is unable to make the payments. This means that if your friend or family member defaults on the loan, you will be on the hook for the remaining balance.There are a few things to consider before co-signing a personal loan for someone:Do you trust the borrower to make the payments on time and in full? If you are not confident that the borrower will be able to make the payments, it may not be a good idea to co-sign the loan.Can you afford to make the payments if the borrower defaults? If you are unable to make the payments, co-signing the loan could put your own financial stability at risk.What is the purpose of the loan? If the borrower is using the loan for a risky or questionable venture, it may not be worth the risk to co-sign.Is there another way for the borrower to get the loan without a co-signer? If the borrower has a good credit score and is able to qualify for a loan on their own, it may not be necessary for you to co-sign.In general, it is important to carefully consider the risks and potential consequences before co-signing a loan for someone. If you do decide to co-sign, it is a good idea to have a conversation with the borrower about their plans for making the loan payments and to have a clear understanding of your responsibilities as a co-signer.",
                "The optimal approach to risk management involves careful assessment of market conditions.",
            ],
            "label": [0, 0, 0, 1, 1, 1],  # 0=human, 1=AI
        }

        df = pd.DataFrame(sample_data)
        logger.info(f"Created sample dataset with {len(df)} examples")
        return df

    def load_origin_dataset(self) -> pd.DataFrame:
        """Load finance dataset from JSON Lines format"""
        files = self.find_finance_data_files()

        logger.info("Loading finance datasets...")
        logger.info(f"Found files: {list(files.keys())}")

        all_texts = []
        all_labels = []

        if "original" in files:
            finance_path = files["original"]
            logger.info(f"Loading from: {finance_path}")

            try:
                with open(finance_path, "r", encoding="utf-8") as f:
                    line_count = 0
                    for line_num, line in enumerate(f, 1):
                        line = line.strip()
                        if not line:
                            continue

                        try:
                            data = json.loads(line)
                            line_count += 1

                            # Extract human answers
                            human_answers = data.get("human_answers", [])
                            if (
                                human_answers
                                and len(human_answers) > 0
                                and human_answers[0]
                            ):
                                all_texts.append(human_answers[0])
                                all_labels.append(0)  # 0 for human

                            # Extract ChatGPT answers
                            chatgpt_answers = data.get("chatgpt_answers", [])
                            if (
                                chatgpt_answers
                                and len(chatgpt_answers) > 0
                                and chatgpt_answers[0]
                            ):
                                all_texts.append(chatgpt_answers[0])
                                all_labels.append(1)  # 1 for ChatGPT

                            # Log progress every 1000 lines
                            if line_num % 1000 == 0:
                                logger.info(
                                    f"Processed {line_num} lines, extracted {len(all_texts)} texts"
                                )

                        except json.JSONDecodeError as e:
                            logger.error(f"Error parsing line {line_num}: {e}")
                            logger.error(f"Line content: {line[:100]}...")
                            continue

                logger.info(f"Successfully loaded {line_count} JSON objects")
                logger.info(f"Total texts extracted: {len(all_texts)}")

            except FileNotFoundError:
                logger.error(f"File not found: {finance_path}")
                return self.create_sample_dataset()
            except Exception as e:
                logger.error(f"Error loading finance data: {e}")
                return self.create_sample_dataset()
        else:
            logger.warning("No finance data file found. Creating sample dataset...")
            return self.create_sample_dataset()

        if len(all_texts) == 0:
            logger.warning(
                "No texts extracted from finance data. Creating sample dataset..."
            )
            return self.create_sample_dataset()

        # Create DataFrame
        df = pd.DataFrame(
            {"id": range(len(all_texts)), "text": all_texts, "label": all_labels}
        )

        # Shuffle data
        df = df.sample(frac=1, random_state=42).reset_index(drop=True)
        df["id"] = range(len(df))

        # Print summary
        logger.info(f"\n=== Dataset Summary ===")
        logger.info(f"Total samples: {len(df)}")
        logger.info(f"Human texts: {len(df[df['label'] == 0])}")
        logger.info(f"ChatGPT texts: {len(df[df['label'] == 1])}")

        # Show balance
        balance = len(df[df["label"] == 0]) / len(df) * 100
        logger.info(
            f"Dataset balance: {balance:.1f}% human, {100-balance:.1f}% ChatGPT"
        )

        # Display samples
        if len(df) > 0:
            logger.info("\nSample texts:")
            for label in [0, 1]:
                label_name = "Human" if label == 0 else "ChatGPT"
                samples = df[df["label"] == label]
                if len(samples) > 0:
                    sample_text = samples.iloc[0]["text"]
                    logger.info(f"\n[{label_name}]: {sample_text[:200]}...")

        return df[["id", "text", "label"]]

    def save_dataset(self, df: pd.DataFrame, filename: str, format: str = "csv"):
        """
        Saving a dataset to a file

        Args:
            df: the DataFrame to be saved
            filename: the name of the file (without extension)
            format: file format ('csv', 'json', 'jsonl')
        """
        filepath = os.path.join(self.data_dir, f"{filename}.{format}")

        if format == "csv":
            df.to_csv(filepath, index=False)
        elif format == "json":
            df.to_json(filepath, orient="records", indent=2)
        elif format == "jsonl":
            df.to_json(filepath, orient="records", lines=True)
        else:
            raise ValueError(f"Unsupported format: {format}")

        logger.info(f"Saved dataset to {filepath}")
        return filepath


def load_data(path: str) -> pd.DataFrame:

    loader = CustomDataLoader()
    return loader.load_data(path)


def load_finance_dataset() -> pd.DataFrame:

    loader = CustomDataLoader()
    return loader.load_finance_dataset()


def load_origin_dataset() -> pd.DataFrame:
    loader = CustomDataLoader()
    return loader.load_origin_dataset


def create_sample_dataset() -> pd.DataFrame:

    loader = CustomDataLoader()
    return loader.create_sample_dataset()


def evaluate_detector(
    detector: AITextDetector, texts: List[str], labels: List[int]
) -> Dict[str, float]:
    """
    Evaluating detector performance
    """
    results = detector.detect_batch(texts, labels)

    predictions = results["predictions"]
    probabilities = results["probabilities"]

    metrics = {
        "accuracy": accuracy_score(labels, predictions),
        "auc": roc_auc_score(labels, probabilities),
    }

    print("\n=== Evaluation Results ===")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"AUC: {metrics['auc']:.4f}")

    print("\nClassification Report:")
    print(classification_report(labels, predictions, target_names=["Human", "AI"]))

    human_scores = results["similarity_scores"][np.array(labels) == 0]
    ai_scores = results["similarity_scores"][np.array(labels) == 1]

    print(f"\nSimilarity Score Distribution:")
    print(
        f"Human texts - Mean: {np.mean(human_scores):.3f}, Std: {np.std(human_scores):.3f}"
    )
    print(f"AI texts - Mean: {np.mean(ai_scores):.3f}, Std: {np.std(ai_scores):.3f}")

    return metrics


def plot_roc_curves(results: Dict):

    plt.figure(figsize=(10, 8))
    for name, result_data in results.items():
        fpr, tpr, _ = roc_curve(
            result_data["test_labels"], result_data["test_results"]["probabilities"]
        )
        auc = roc_auc_score(
            result_data["test_labels"], result_data["test_results"]["probabilities"]
        )
        plt.plot(fpr, tpr, label=f"{name} (AUC = {auc:.3f})")
    plt.plot([0, 1], [0, 1], "k--", label="Chance")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curves Comparison")
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_precision_recall_curves(results: Dict):

    plt.figure(figsize=(10, 8))
    for name, result_data in results.items():
        precision, recall, _ = precision_recall_curve(
            result_data["test_labels"], result_data["test_results"]["probabilities"]
        )
        plt.plot(recall, precision, label=f"{name}")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curves Comparison")
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_confusion_matrices(results: Dict):

    n_results = len(results)
    fig, axes = plt.subplots(1, n_results, figsize=(6 * n_results, 5))
    if n_results == 1:
        axes = [axes]  # Make it iterable
    for ax, (name, result_data) in zip(axes, results.items()):
        cm = confusion_matrix(
            result_data["test_labels"], result_data["test_results"]["predictions"]
        )
        sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            ax=ax,
            cbar=False,
            xticklabels=["Human", "AI"],
            yticklabels=["Human", "AI"],
        )
        ax.set_title(f"Confusion Matrix: {name}")
        ax.set_xlabel("Predicted Label")
        ax.set_ylabel("True Label")
    plt.tight_layout()
    plt.show()


def plot_classification_metrics(results: Dict):

    metrics_data = []
    for name, result_data in results.items():
        report = classification_report(
            result_data["test_labels"],
            result_data["test_results"]["predictions"],
            target_names=["Human", "AI"],
            output_dict=True,
        )
        ai_metrics = report["AI"]
        metrics_data.append(
            {
                "Configuration": name,
                "Precision": ai_metrics["precision"],
                "Recall": ai_metrics["recall"],
                "F1-Score": ai_metrics["f1-score"],
            }
        )

    df = pd.DataFrame(metrics_data)
    df_melted = df.melt(id_vars="Configuration", var_name="Metric", value_name="Score")

    plt.figure(figsize=(12, 7))
    sns.barplot(data=df_melted, x="Configuration", y="Score", hue="Metric")
    plt.title('Classification Metrics for "AI" Class')
    plt.ylabel("Score")
    plt.xticks(rotation=15)
    plt.ylim(0, 1.05)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()


def find_optimal_threshold(labels: List[int], scores: np.ndarray) -> float:
    """
    Find the optimal classification threshold using the ROC curve.

    The optimal threshold is the one that maximizes the difference
    between the true positive rate (TPR) and the false positive rate (FPR).
    """
    fpr, tpr, thresholds = roc_curve(labels, scores)

    # Calculate the optimal threshold
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    print(
        f"\nFound optimal similarity threshold from training data: {optimal_threshold:.4f}"
    )
    return optimal_threshold



async def main(
    data_path: str = "finance",
    max_samples: Optional[int] = None,
    use_cache: bool = True,
    config: Optional[DetectionConfig] = None,
):

    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(SEED)

    print("=== Enhanced AI Text Detection ===")
    print(
        f"Method: Multi-feature extraction with {config.revision_model if config else 'default model'}"
    )

    logger.info(f"Loading data from: {data_path}")
    loader = CustomDataLoader()
    df = loader.load_data(data_path)
    if max_samples:
        df = df.head(max_samples)
        print(f"Using first {max_samples} samples for testing")

    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    texts = df["text"].tolist()
    labels = df["label"].tolist()

    # Data segmentation
    split_idx = int(len(texts) * 0.6)
    train_texts, test_texts = texts[:split_idx], texts[split_idx:]
    train_labels, test_labels = labels[:split_idx], labels[split_idx:]

    print(f"\nDataset split: {len(train_texts)} training, {len(test_texts)} testing")

    # Creating Detectors
    if config is None:
        config = DetectionConfig(
            revision_model="t5-small",  # 或 "gpt2" t5-small
            embedding_model="all-MiniLM-L6-v2",
            perturbation_rate=0.15,
            use_ml_classifier=True,
        )

    detector = AITextDetector(config)

    # Load Cache
    cache_path = f"enhanced_cache_{config.revision_model}.json"
    if use_cache and os.path.exists(cache_path):
        with open(cache_path, "r") as f:
            detector.cache = json.load(f)
        print(f"Loaded cache with {len(detector.cache)} entries")

    # training phase
    print("\n=== Training Phase ===")
    print(
        "Train labels distribution:", pd.Series(train_labels).value_counts().to_dict()
    )
    train_results = await detector.detect_batch(train_texts, train_labels)
    # If not using the ML classifier, find the best threshold from the training run
    if not config.use_ml_classifier:
        optimal_threshold = find_optimal_threshold(
            train_labels, train_results["similarity_scores"]
        )
        # Update the detector's config with the new, optimal threshold
        detector.config.similarity_threshold = optimal_threshold

    # testing phase
    print("\n=== Testing Phase ===")
    test_results = await detector.detect_batch(test_texts)

    # Assessment results
    test_predictions = test_results["predictions"]
    test_probabilities = test_results["probabilities"]

    # Calculation of indicators
    accuracy = accuracy_score(test_labels, test_predictions)
    auc = roc_auc_score(test_labels, test_probabilities)
    f1 = f1_score(test_labels, test_predictions)

    print(f"\n=== Test Results ===")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"F1 Scores: {f1:.4f}")
    # Detailed report
    print("\nClassification Report:")
    print(
        classification_report(
            test_labels, test_predictions, target_names=["Human", "AI"]
        )
    )

    # confusion matrix (math.)
    cm = confusion_matrix(test_labels, test_predictions)
    print("\nConfusion Matrix:")
    print(f"              Predicted")
    print(f"            Human    AI")
    print(f"Actual Human  {cm[0,0]:4d}  {cm[0,1]:4d}")
    print(f"       AI     {cm[1,0]:4d}  {cm[1,1]:4d}")

    # character analysis
    features_df = test_results["features"]
    similarity_scores = test_results["similarity_scores"]

    # Subgroup statistics
    human_scores = similarity_scores[np.array(test_labels) == 0]
    ai_scores = similarity_scores[np.array(test_labels) == 1]

    print(f"\n=== Similarity Score Analysis ===")
    print(f"Human texts:")
    print(f"  Mean: {np.mean(human_scores):.3f}, Std: {np.std(human_scores):.3f}")
    print(f"  Min: {np.min(human_scores):.3f}, Max: {np.max(human_scores):.3f}")

    print(f"AI texts:")
    print(f"  Mean: {np.mean(ai_scores):.3f}, Std: {np.std(ai_scores):.3f}")
    print(f"  Min: {np.min(ai_scores):.3f}, Max: {np.max(ai_scores):.3f}")

    print("\n=== Misclassified Examples ===")
    misclassified_idx = np.where(test_predictions != test_labels)[0]

    for i, idx in enumerate(misclassified_idx[:50]):  # 显示前5个
        true_label = "AI" if test_labels[idx] == 1 else "Human"
        pred_label = "AI" if test_predictions[idx] == 1 else "Human"
        text = (
            test_texts[idx][:100] + "..."
            if len(test_texts[idx]) > 100
            else test_texts[idx]
        )

        print(f"\nExample {i+1}:")
        print(f"Text: {text}")
        print(f"True: {true_label}, Predicted: {pred_label}")
        print(
            f"Similarity: {similarity_scores[idx]:.3f}, Probability: {test_probabilities[idx]:.3f}"
        )

    if use_cache:
        with open(cache_path, "w") as f:
            json.dump(detector.cache, f)
        print(f"\nCache saved to {cache_path}")

    model_path = f"enhanced_detector_{config.revision_model}.pkl"
    detector.save_model(model_path)

    return {
        "detector": detector,
        "test_results": test_results,
        "test_labels": test_labels,
        "accuracy": accuracy,
        "auc": auc,
        "f1": f1,
    }




NLTK data download completed.


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\suzehao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from typing import List, Dict, Tuple, Optional
import pandas as pd
from tqdm import tqdm


class DualPathFusionModel(nn.Module):
  

    def __init__(
        self,
        transformer_name: str = "distilbert-base-uncased",
        num_statistical_features: int = 13,
        hidden_size: int = 256,
        dropout_rate: float = 0.3,
    ):
   
        super(DualPathFusionModel, self).__init__()

        
        self.transformer = AutoModel.from_pretrained(transformer_name)
        self.transformer_dim = self.transformer.config.hidden_size

        # Freeze the first few layers of the Transformer
        for param in self.transformer.base_model.embeddings.parameters():
            param.requires_grad = False

        # Path B uses the input statistical features directly

        # Classification head after feature fusion
        fusion_dim = self.transformer_dim + num_statistical_features

        self.classifier = nn.Sequential(
            nn.Linear(fusion_dim, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size // 2, 2),  # binary classification
        )

        # feature normalisation layer
        self.feature_norm = nn.BatchNorm1d(num_statistical_features)

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        statistical_features: torch.Tensor,
    ) -> torch.Tensor:
        
        # Path A: Extract deep semantic features via Transformer
        transformer_outputs = self.transformer(
            input_ids=input_ids, attention_mask=attention_mask
        )

        # Take the output of the [CLS] token as a sentence representation
        cls_embedding = transformer_outputs.last_hidden_state[
            :, 0, :
        ]  # [batch_size, transformer_dim]

        # Path B: Normalised Statistical Features
        normalized_features = self.feature_norm(
            statistical_features
        )  # [batch_size, num_features]

        # Feature fusion: stitching two features
        fused_features = torch.cat([cls_embedding, normalized_features], dim=1)

        # The final prediction is obtained by sorting the header
        logits = self.classifier(fused_features)

        return logits


class TextDetectionDataset(Dataset):
    """
    AI Text Detection Dataset
    """

    def __init__(
        self,
        texts: List[str],
        statistical_features: np.ndarray,
        labels: List[int],
        tokenizer,
        max_length: int = 512,
    ):
        """
        Args.
            texts: list of original texts
            statistical_features: matrix of statistical features [n_samples, n_features]
            labels: list of labels (0=human, 1=AI)
            tokenizer: Transformer lexicon
            max_length: Maximum sequence length
        """
        self.texts = texts
        self.statistical_features = statistical_features
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        
        text = self.texts[idx]
        label = self.labels[idx]
        features = self.statistical_features[idx]

      
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "statistical_features": torch.tensor(features, dtype=torch.float32),
            "labels": torch.tensor(label, dtype=torch.long),
        }


class DualPathDetector:
    """
    Detectors using dual path fusion models
    """

    def __init__(self, model_name: str = "distilbert-base-uncased", device: str = None):
        """
        Initialising the detector
        """
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model_name = model_name

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Initialise model (will be adjusted later for number of features)
        self.model = None
        self.scaler = StandardScaler()

        print(f"Dual Path Detector initialized on {self.device}")

    def prepare_data(
        self, texts: List[str], features_df: pd.DataFrame, labels: List[int]
    ) -> Tuple[Dataset, Dataset]:
        """
        Preparing training and validation data

        Args.
            texts: list of texts
            features_df: DataFrame with statistical features
            labels: List of labels

        Returns.
            train_dataset, val_dataset
        """
        # Standardised features
        features_array = features_df.values
        features_normalized = self.scaler.fit_transform(features_array)

        # Split data
        X_train, X_val, y_train, y_val, feat_train, feat_val = train_test_split(
            texts,
            labels,
            features_normalized,
            test_size=0.2,
            random_state=42,
            stratify=labels,
        )

        # Creating a Data Set
        train_dataset = TextDetectionDataset(
            X_train, feat_train, y_train, self.tokenizer
        )
        val_dataset = TextDetectionDataset(X_val, feat_val, y_val, self.tokenizer)

        return train_dataset, val_dataset

    def train(
        self,
        train_dataset: Dataset,
        val_dataset: Dataset,
        num_epochs: int = 10,
        batch_size: int = 16,
        learning_rate: float = 2e-5,
        save_path : str = "best_dual_path_model.pt"
    ):
        """
        Training Models
        """
        # If the model has not been initialised, initialise based on the number of features
        if self.model is None:
            num_features = train_dataset[0]["statistical_features"].shape[0]
            self.model = DualPathFusionModel(
                transformer_name=self.model_name, num_statistical_features=num_features
            ).to(self.device)

        # Create the data loader
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        # Optimiser and loss function
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()

        # Learning rate scheduler
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode="min", patience=2, factor=0.5
        )

        # Training cycles
        best_val_loss = float("inf")

        for epoch in range(num_epochs):
            # Training phase
            self.model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0

            train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
            for batch in train_bar:
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                features = batch["statistical_features"].to(self.device)
                labels = batch["labels"].to(self.device)

                optimizer.zero_grad()

                # Forward propagation
                logits = self.model(input_ids, attention_mask, features)
                loss = criterion(logits, labels)

                # Reverse propagation
                loss.backward()
                optimizer.step()

                # Statistics
                train_loss += loss.item()
                _, predicted = torch.max(logits.data, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()

                train_bar.set_postfix(
                    {
                        "loss": f"{loss.item():.4f}",
                        "acc": f"{100.*train_correct/train_total:.2f}%",
                    }
                )

            # validation phase
            self.model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for batch in tqdm(
                    val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"
                ):
                    input_ids = batch["input_ids"].to(self.device)
                    attention_mask = batch["attention_mask"].to(self.device)
                    features = batch["statistical_features"].to(self.device)
                    labels = batch["labels"].to(self.device)

                    logits = self.model(input_ids, attention_mask, features)
                    loss = criterion(logits, labels)

                    val_loss += loss.item()
                    _, predicted = torch.max(logits.data, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()

            # Calculation of average loss and accuracy
            avg_train_loss = train_loss / len(train_loader)
            avg_val_loss = val_loss / len(val_loader)
            train_acc = 100.0 * train_correct / train_total
            val_acc = 100.0 * val_correct / val_total

            print(f"\nEpoch {epoch+1}:")
            print(f"  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.2f}%")
            print(f"  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.2f}%")

            # Learning rate scheduling
            scheduler.step(avg_val_loss)

            
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                self.save_model(save_path)
                print("   Saved best modelto {save_path}")




    def predict(
        self, texts: List[str], features_df: pd.DataFrame
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Predict new text

        Returns.
            predictions: prediction labels
            probabilities: Predicted probabilities
        """
        self.model.eval()

        
        features_normalized = self.scaler.transform(features_df.values)

     
        dataset = TextDetectionDataset(
            texts, features_normalized, [0] * len(texts), self.tokenizer  # false label
        )
        loader = DataLoader(dataset, batch_size=32)

        all_predictions = []
        all_probabilities = []

        with torch.no_grad():
            for batch in tqdm(loader, desc="Predicting"):
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                features = batch["statistical_features"].to(self.device)

                logits = self.model(input_ids, attention_mask, features)
                probs = F.softmax(logits, dim=1)

                predictions = torch.argmax(logits, dim=1)

                all_predictions.extend(predictions.cpu().numpy())
                all_probabilities.extend(probs[:, 1].cpu().numpy())  # AI类的概率

        return np.array(all_predictions), np.array(all_probabilities)

    def save_model(self, path: str):
        
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "scaler": self.scaler,
                "model_name": self.model_name,
                "num_features": self.model.classifier[0].in_features
                - self.model.transformer_dim,
            },
            path,
        )

    def load_model(self, path: str):
        
        checkpoint = torch.load(path, map_location=self.device)

 
        self.model = DualPathFusionModel(
            transformer_name=checkpoint["model_name"],
            num_statistical_features=checkpoint["num_features"],
        ).to(self.device)

        self.model.load_state_dict(checkpoint["model_state_dict"])
        self.scaler = checkpoint["scaler"]
        self.model.eval()



In [ ]:


"""
Integrating Dual Path Fusion Models into Existing AI Text Detection Processes
"""
import sys
import torch
import numpy as np
import pandas as pd
from typing import Dict, List, Optional



class EnhancedAIDetector:
    """
    MODIFIED AI Detector: Integrates the dual-path model.
    The __init__ method is updated to accept a model name for the dual-path detector.
    """
    def __init__(self, config: Optional[DetectionConfig] = None, dual_path_model_name: str = "distilbert-base-uncased"):
        """
        Args:
            config: Configuration for the base detector (feature extraction).
            dual_path_model_name: The name of the transformer to use in the DualPathFusionModel.
        """
        self.config = config or DetectionConfig()
        self.base_detector = AITextDetector(self.config)
        # Pass the dynamic model name to the dual-path detector
        self.dual_path_detector = DualPathDetector(model_name=dual_path_model_name)
        print(f"Enhanced AI Detector initialized with Dual Path Model: {dual_path_model_name}")

    
    async def extract_all_features(self, texts: List[str]) -> pd.DataFrame:
        
        print("Extracting statistical features...")
        results = await self.base_detector.detect_batch(texts)
        features_array = results['features']
     
        all_possible_features = [
            'semantic_similarity', 'word_overlap_ratio', 'vocabulary_change_ratio',
            'unique_words_ratio', 'stopword_ratio_change', 'sentence_count_ratio',
            'avg_sentence_length_change', 'pos_NN_ratio_change', 'pos_VB_ratio_change',
            'pos_JJ_ratio_change', 'pos_RB_ratio_change', 'punctuation_ratio_change',
            'capitalization_ratio_change', 'avg_word_length_change', 'char_level_similarity',
            'word_level_similarity', 'length_ratio'
        ]
        core_features = all_possible_features[:13] 
        features_df = pd.DataFrame(features_array[:, :13], columns=core_features)
        return features_df

    async def train_dual_path_model(self, train_df: pd.DataFrame, num_epochs: int, batch_size: int, save_path: str):
        print("=== Training Dual Path Model ===")
        texts = train_df["text"].tolist()
        labels = train_df["label"].tolist()
        print("1. Extracting features...")
        features_df = await self.extract_all_features(texts)
        print("2. Preparing datasets...")
        train_dataset, val_dataset = self.dual_path_detector.prepare_data(texts, features_df, labels)
        print("3. Training model...")
        self.dual_path_detector.train(train_dataset, val_dataset, num_epochs=num_epochs, batch_size=batch_size, save_path=save_path)
        print("\n Training completed!")

    async def detect_with_dual_path(self, texts: List[str]) -> Dict:
        features_df = await self.extract_all_features(texts)
        predictions, probabilities = self.dual_path_detector.predict(texts, features_df)
        return {"predictions": predictions, "probabilities": probabilities, "features": features_df}

def setup_logging(output_dir: str = "./result"):
    """
    Sets up logging to save all print output to a file and also show it on the console.
    """
    log_file_path = os.path.join(output_dir, 'output.log')
    os.makedirs(output_dir, exist_ok=True)

    # Get the root logger
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # Clear existing handlers to avoid duplicate logs in interactive environments
    if logger.hasHandlers():
        logger.handlers.clear()


    file_handler = logging.FileHandler(log_file_path, mode='w', encoding='utf-8')
    file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(file_formatter)
    logger.addHandler(file_handler)


    console_handler = logging.StreamHandler(sys.stdout)
    console_formatter = logging.Formatter('%(message)s')
    console_handler.setFormatter(console_formatter)
    logger.addHandler(console_handler)

    logging.info(f"Logging setup complete. All output will be saved to {log_file_path}")

def plot_roc_curves(results: Dict, output_dir: str = "./result"):
    
    plt.figure(figsize=(10, 8))
    for name, result_data in results.items():
        fpr, tpr, _ = roc_curve(result_data["test_labels"], result_data["test_results"]["probabilities"])
        auc = roc_auc_score(result_data["test_labels"], result_data["test_results"]["probabilities"])
        plt.plot(fpr, tpr, label=f"{name} (AUC = {auc:.3f})")
    plt.plot([0, 1], [0, 1], "k--", label="Chance")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curves Comparison")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, "roc_curves_comparison.png"))
    plt.show()

def plot_precision_recall_curves(results: Dict, output_dir: str = "./result"):
    
    plt.figure(figsize=(10, 8))
    for name, result_data in results.items():
        precision, recall, _ = precision_recall_curve(result_data["test_labels"], result_data["test_results"]["probabilities"])
        plt.plot(recall, precision, label=f"{name}")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curves Comparison")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, "precision_recall_curves_comparison.png"))
    plt.show()

def plot_confusion_matrices(results: Dict, output_dir: str = "./result"):
    
    n_results = len(results)
    fig, axes = plt.subplots(1, n_results, figsize=(6 * n_results, 5), squeeze=False)
    for ax, (name, result_data) in zip(axes.flatten(), results.items()):
        cm = confusion_matrix(result_data["test_labels"], result_data["test_results"]["predictions"])
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax, cbar=False, xticklabels=["Human", "AI"], yticklabels=["Human", "AI"])
        ax.set_title(f"Confusion Matrix: {name}")
        ax.set_xlabel("Predicted Label")
        ax.set_ylabel("True Label")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrices_comparison.png"))
    plt.show()

def plot_classification_metrics(results: Dict, output_dir: str = "./result"):
    
    metrics_data = []
    for name, result_data in results.items():
        report = classification_report(result_data["test_labels"], result_data["test_results"]["predictions"], target_names=["Human", "AI"], output_dict=True)
        ai_metrics = report["AI"]
        metrics_data.append({"Configuration": name, "Precision": ai_metrics["precision"], "Recall": ai_metrics["recall"], "F1-Score": ai_metrics["f1-score"]})
    df = pd.DataFrame(metrics_data)
    df_melted = df.melt(id_vars="Configuration", var_name="Metric", value_name="Score")
    plt.figure(figsize=(12, 7))
    sns.barplot(data=df_melted, x="Configuration", y="Score", hue="Metric")
    plt.title('Classification Metrics for "AI" Class')
    plt.ylabel("Score")
    plt.xticks(rotation=15)
    plt.ylim(0, 1.05)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.savefig(os.path.join(output_dir, "classification_metrics_comparison.png"))
    plt.show()


async def main_workflow_for_comparison(
    data_path: str,
    max_samples: Optional[int],
    use_cache: bool,
    config: DetectionConfig,
    dual_path_model_name: str,
    save_path: str 
):
    """
    MODIFIED Main Workflow: This function now uses logging instead of print.
    """
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(SEED)

    detector = EnhancedAIDetector(config=config, dual_path_model_name=dual_path_model_name)

    cache_path = f"enhanced_cache_{config.revision_model}.json"
    if use_cache and os.path.exists(cache_path):
        try:
            with open(cache_path, "r") as f:
                detector.base_detector.cache = json.load(f)
            logging.info(f"Successfully loaded cache with {len(detector.base_detector.cache)} entries from {cache_path}")
        except Exception as e:
            logging.warning(f"Could not load cache from {cache_path}: {e}")
    elif use_cache:
        logging.info(f"Cache file not found at {cache_path}. A new cache will be created.")

    loader = CustomDataLoader()
    df_full = loader.load_data(data_path)
    df = df_full.head(max_samples) if max_samples else df_full
    
    train_df, test_df = train_test_split(df, test_size=0.3, random_state=SEED, stratify=df['label'])
    logging.info(f"Data prepared: {len(train_df)} training, {len(test_df)} testing.")

    await detector.train_dual_path_model(train_df=train_df, num_epochs=5, batch_size=16, save_path=save_path)

    test_texts = test_df["text"].tolist()
    test_labels = test_df["label"].tolist()
    test_results = await detector.detect_with_dual_path(test_texts)


    accuracy = accuracy_score(test_labels, test_results['predictions'])
    auc = roc_auc_score(test_labels, test_results['probabilities'])
    report_dict = classification_report(test_labels, test_results['predictions'], output_dict=True)
    f1_score = report_dict['weighted avg']['f1-score']

    logging.info(f"\n--- Test Results for {dual_path_model_name} ---")
    logging.info(f"Accuracy: {accuracy:.4f}")
    logging.info(f"AUC: {auc:.4f}")
    logging.info(f"Weighted F1-Score: {f1_score:.4f}")
    logging.info("\nClassification Report:")
    logging.info(f"\n{classification_report(test_labels, test_results['predictions'], target_names=['Human', 'AI'], digits=4)}")

    return {"detector": detector, "test_results": test_results, "test_labels": test_labels, "accuracy": accuracy, "auc": auc, "f1_score": f1_score}




async def run_transformer_comparison(data_path: str = "finance", max_samples: Optional[int] = 1000):
    """
    Runs a comparison experiment between different transformer models for both
    feature extraction and deep feature representation.
    """
    setup_logging() # Setup logging to file and console
    logging.info("=== Transformer Model Comparison Experiment ===")
    results = {}

    # Define the combinations of models you want to test
    configurations = {
        # "BERT": {
        #     "embedding_model": "bert-base-uncased",
        #     "transformer_name": "bert-base-uncased"
        # },
        # "DistilBERT": {
        #     "embedding_model": "distilbert-base-uncased",
        #     "transformer_name": "distilbert-base-uncased"
        # },
        # "fine-tune model": {
        #     "embedding_model": "roberta-base",
        #     "transformer_name": "roberta-base"
        # },
        # "RoBERTa":{
        #     "embedding_model": "sentence-transformers/paraphrase-MiniLM-L6-v2",
        #     "transformer_name": "sentence-transformers/paraphrase-MiniLM-L6-v2",
        # },
        # "RoBERTa":{
        #     "embedding_model": "./models/paraphrase-MiniLM-L6-v2-ai-detector-incomplete",
        #     "transformer_name": "./models/paraphrase-MiniLM-L6-v2-ai-detector-incomplete",
        # },
        # "T5": {
        #     "embedding_model": "t5-small",
        #     "transformer_name": "t5-small"
        # },
        # "GPT-2": {
        #     "embedding_model": "gpt2",
        #     "transformer_name": "gpt2"
        # },
        "fine-tune model": {
            "embedding_model": "./models/paraphrase-MiniLM-L6-v2-ai-detector-incomplete",
            "transformer_name": "roberta-base"
        },
        
    }

   
    for name, model_config in configurations.items():
        logging.info(f"\n{'='*25} Running Experiment: {name} {'='*25}")
        try:
            model_save_path = f"best_model_{name.replace(' ', '_').replace('/', '-')}.pt"

            detection_config = DetectionConfig(
                revision_model="gpt-3.5-turbo",
                embedding_model=model_config["embedding_model"],
                perturbation_rate=0.15,
                use_ml_classifier=True,
            )
            
            results[name] = await main_workflow_for_comparison(
                data_path=data_path,
                max_samples=max_samples,
                use_cache=True,
                config=detection_config,
                dual_path_model_name=model_config["transformer_name"],
                save_path=model_save_path 
            )
        except Exception as e:
            import traceback
            logging.error(f"\n!!!!!! Experiment '{name}' FAILED: {e} !!!!!!")
            logging.error(traceback.format_exc())

    logging.info("\n\n" + "="*25 + " Final Comparison Summary " + "="*25)
    summary_df = pd.DataFrame([
        {"Configuration": name, "Accuracy": res["accuracy"], "AUC": res["auc"], "F1-Score": res["f1_score"]}
        for name, res in results.items()
    ])
    summary_df['Accuracy'] = summary_df['Accuracy'].map('{:.4f}'.format)
    summary_df['AUC'] = summary_df['AUC'].map('{:.4f}'.format)
    summary_df['F1-Score'] = summary_df['F1-Score'].map('{:.4f}'.format)
    logging.info(f"\n{summary_df.to_string(index=False)}")

    logging.info("\n\nGenerating visualizations...")
    if results:
        plot_roc_curves(results)
        plot_precision_recall_curves(results)
        plot_confusion_matrices(results)
        plot_classification_metrics(results)
    else:
        logging.warning("No successful experiments were completed to visualize.")

    return results

if __name__ == "__main__":
    
    await run_transformer_comparison(data_path="test", max_samples=None)

    

Logging setup complete. All output will be saved to ./result\output.log
=== Transformer Model Comparison Experiment ===

========================= Running Experiment: fine-tune model =========================


The following layers were not sharded: distilbert.transformer.layer.*.attention.out_lin.weight, distilbert.transformer.layer.*.attention.v_lin.bias, distilbert.embeddings.position_embeddings.weight, distilbert.transformer.layer.*.output_layer_norm.bias, vocab_transform.bias, distilbert.transformer.layer.*.ffn.lin*.bias, distilbert.transformer.layer.*.attention.k_lin.bias, vocab_layer_norm.bias, distilbert.transformer.layer.*.sa_layer_norm.bias, vocab_projector.weight, distilbert.transformer.layer.*.attention.q_lin.weight, distilbert.transformer.layer.*.attention.out_lin.bias, distilbert.embeddings.LayerNorm.weight, distilbert.transformer.layer.*.sa_layer_norm.weight, distilbert.transformer.layer.*.ffn.lin*.weight, distilbert.transformer.layer.*.attention.v_lin.weight, vocab_transform.weight, vocab_layer_norm.weight, distilbert.embeddings.word_embeddings.weight, distilbert.transformer.layer.*.attention.k_lin.weight, distilbert.embeddings.LayerNorm.bias, distilbert.transformer.layer.*.ou

OpenAI client initialized for model: gpt-3.5-turbo
Using custom base URL: https://api.chatanywhere.tech/v1
Use pytorch device_name: cuda:0
Load pretrained SentenceTransformer: ./models/paraphrase-MiniLM-L6-v2-ai-detector-incomplete


The following layers were not sharded: embeddings.token_type_embeddings.weight, encoder.layer.*.intermediate.dense.bias, pooler.dense.weight, encoder.layer.*.output.dense.bias, encoder.layer.*.attention.self.query.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key.bias, encoder.layer.*.attention.self.value.bias, embeddings.word_embeddings.weight, pooler.dense.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias, encoder.layer.*.attention.self.value.weight, embeddings.position_embeddings.weight, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.attention.self.query.bias, encoder.layer.*.attention.self.key.weight, encoder.layer.*.output.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.attention.output.dense.weight



--- Parameters for Embedding Model: ./models/paraphrase-MiniLM-L6-v2-ai-detector-incomplete ---
SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)
Total Trainable Parameters: 22,713,216
Dual Path Detector initialized on cuda
Enhanced AI Detector initialized with Dual Path Model: roberta-base
Successfully loaded cache with 7844 entries from enhanced_cache_gpt-3.5-turbo.json
Searching for finance data files...
Loading finance datasets...
Found files: ['revised_chatgpt', 'revised_human', 'original']
Loading from: ./data\finance.jsonl
Processed 1000 lines, extracted 2000 texts
Processed 2000 lines, extracted 4000 texts
Processed 30

Perturbing:   1%|▏         | 74/5506 [00:21<36:04,  2.51it/s]